
# Discussion of Timing, Stages, Steps, Etc.

(This notebook was created as a point of reference to an email thread on how we should change the timing of our models in HARK.  Below my text is the text of Matt's email (and at the end of that email are a few responses by me)).

In a meeting on 2021-06-24, we agreed that our treatment of the timing of shocks that occur between period $t$ and period $t+1$ should be changed so that the description of the shocks, and all computations involving them, should be viewed as part of the period $t+1$ `solution` object.  

Agreeing to this change constitutes progress on resolving the question of where to draw the boundaries that _require_ construction of separate "solution" objects -- here I am evading whether to describe these as steps, frames, stages, cycles, or whatever -- I'm referring to solution objects as those that are laid out in a sequence (currently the `solution[]` list) in which each successive object is self-contained.

Of course that just raises the question of what "self-contained" means.

At a minimum, the object should contain everything needed to conduct both the solution and the simulation stage of the problem.  Usually, the circumstances under which the simulation process changes will correspond to circumstances under which the solution object changes.  But, I think I've come up with a counterexample which vindicates the requirement that we should keep track of both.

Suppose we are solving the problem by time iteration on policy rules.  

Suppose further that there is some exogenous probability that any particular person becomes "depressed" which is modeled as a discrete jump downward in the LEVEL of their value function, followed by a sequence of further downward steps in value as time progresses.
Because it has no effect on marginal utility (unless the person can change their probability of survival), depression (as described above) has no effect on behavior.  

Nevertheless, suppose the government is trying to decide whether to put Prozac in the water supply.  Determining whether it should or should not do that will require simulating the model's predictions for the level of value, not just behavior.  
Admittedly, this example is artificial.  But it does establish that there might be different criteria for "something about the agent's circumstances has changed" for simulation versus solution.  So, here's a tentative answer: A separate solution object MUST exist whenever an irreversible change to the problem occurs, even if that change does not affect behavior.  (This is a lower bound; it does not rule out construction of finer-grained solution objects).  (Thinking ahead to potential use of the tools for doing ABM modeling, we could imagine other things that do not affect an agent but that affect other agents -- like being an asymptomatic carrier of the Coronavirus.  So we would need to have a state variable to keep track of infection status even if it has no effect on the agent).  

Aside from becoming depressed, a more prosaic example of an irreversible change is getting older by a year.  No 90 year old can make a transition to being a 30 year old.

An example of a reversible change would be an infinite horizon model in which nevertheless each year is divided into "seasons."   In my view we ought NOT to forbid someone from constructing a model in which the 'solution' object embeds the successive solutions to the four seasonal problems - so long as the simulation machinery is also present.  (Indeed, Mateo has done something along these lines in his portfolio model).

Matt seems to be advocating a stronger mandate, which is that successive solution objects must be constructed whenever anything about the problem evolves.  

This takes a good idea too far.  The machinery for constructing a "self-contained" object (with value functions, marginal value functions, a full suite of evaluations of the level and multiple derivatives of the value function, etc etc) is too cumbersome to be always worth the cost.  (Note that I have left out decision rules; this is because we should definitely allow the construction of stages in which no decision is made).  

The addition of portfolio choice to the consumption-saving problem illustrates why this framing could be useful.  

Currently, we have:

$\mathfrak{V}_{t}(a_{t}) = \mathbb{E}_{t}[ \beta (\Gamma \psi_{t+1})^{1-\rho_{t+1}}v_{t+1}(R / (\Gamma \psi_{t+1})a_{t} + \theta_{t+1})]$

(where the $t+1$ on $\rho_{t+1}$ is the reason we can't compute this object from within period $t$ without explicitly knowing $\rho_{t+1}$ in addition to $v_{t+1}$).

The new proposal is essentially to define a new object, whose value function I will notate by using the mathcal font rather than Gothic.  
If we uncouthly define a new $\aleph_{t+1} \equiv a_{t}$ (because aleph is the Hebrew letter corresponding to a), we now just need to drop the $\beta$ (which properly belongs to period $t$) from the expression above to calculate 

$\mathcal{V}_{t+1}(\aleph_{t+1}) = \mathbb{E}_{t}[(\Gamma \psi_{t+1})^{1-\rho_{t+1}}v_{t+1}(R / (\Gamma \psi_{t+1})a_{t} + \theta_{t+1})]$

where our former $\mathfrak{V}_{t}$ is now simply 
$\mathfrak{V}_{t}(a_{t}) = \beta_{t} \mathcal{V}_{t+1}(\aleph_{t+1})$

with the (potentially time-varying) $\beta$ residing properly in period $t$.  
We can construct a grid of $\aleph$ points $\vec{\aleph}$ and compute $\vec{\mathcal{V}}_{t+1}$ and its derivatives.
It will then be trivial (and often necessary, or at least desirable) to construct an end-of-period state with marginal values at a set of end-of-period assets, $\vec{\mathfrak{V}}^{\prime}_{t} \equiv \mathfrak{V}_{t}^{\prime}(\vec{a}) = \beta_{t} \mathcal{V}_{t+1}^{\prime}(\vec{\aleph})$, and we will want to use the same gridpoints for $a_{t}$ as for $\aleph_{t+1}$ 

In the basic consumption-saving problem EGM translates these points translate directly to levels of consumption via
$\vec{c}_{t} = \left(\vec{\mathfrak{V}}_{t}^{\prime}\right)^{-1/\rho}$
Now suppose we want to add portfolio choice to the problem. 
This requires two steps:
1. We need to allow for risky returns as another stochastic variable realized in period $t+1$.  Call those returns $\tilde{R}_{t+1}$.
1. We need to allow for portfolio choice before the realizations of the risky shocks that occur at the beginning of $t+1$.

Suppose the share of the portfolio in the risky asset is $\zeta$.  Then the realized weighted return will be:

$\mathbb{R}_{t+1}=(1-\zeta) R + \zeta \tilde{R}_{t+1}$  

Unhappily, this would seem to imply that we need to add the portfolio allocation as another state variable at the beginning of $t+1$:

$\mathcal{V}_{t+1}(\aleph_{t+1},\zeta)$ 

because the optimal portfolio choice $\zeta^{*}$ will satisfy the first order condition 

$\mathbb{E}_{t}[(\mathbb{R}_{t+1}-R)v^{\prime}_{t+1}]$

so if we want to be consistent in our decision to segregate computations to the period in which the relevant shocks are defined, now it seems that we need to have the portfolio choice as a state variable at the beginning of period $t+1$; defining a new "Whole" value function $W$:

$\mathcal{W}_{t+1}(\aleph_{t+1}, \zeta)$.  

But here's the key point:  Suppose we actually solve the portfolio choice problem at the beginning of $t+1$, yielding $\zeta^{*}_{t+1}$.  Then we can substitute the optimized portfolio share to obtain a new value function $\mathcal{X}$:

$\mathcal{X}_{t+1}(\aleph_{t+1}) = \mathcal{W}(\aleph_{t+1},\zeta^{*}_{t+1})$ 

which means that actually the only state variable we need to carry over between $t$ and $t+1$ is $\aleph_{t+1}=a_{t}$.  Furthermore, the _exact same code_ that solved the plain consumption-saving program without portfolio choice now solves the problem with portfolio choice, because we now have:

$\mathfrak{V}_{t}(a_{t}) = \beta_{t} \mathcal{X}(\aleph_{t+1})$ and the only thing we need to solve the problem is the same $\vec{\mathfrak{V}}_{t}^{\prime}$ as before (though it will have different values at the same $a$ gridpoints because of the portfolio choice).

This has all taken much more writing to explain than I anticipated.  But the bottom line is that with this way of doing things the portfolio choice aspect of the problem is _completely modular_.  No modification needs to be made to the tiniest part of the existing solution for the problem with a riskfree rate; the portfolio agent will just use that code as-is, calculating an expected return over appropriately chosen values of $R$.  (Of course, this only works under the standard assumption that the risky return shock is IID). 

A (computationally; not intuitively) natural approach is, in the portfolio stage, to

1. Make a grid of potential portfolio share choices; say, `(0,0.1,...0.9,1.0)` 
1. For each grid, construct the corresponding distribution of the portfolio return $\mathbb{R}_{t+1}$ under some assumption about the structure of the risky returns $\tilde{R}_{t+1}$.
1. Then for each gridpoint, 
    *  Construct expectations for  $v_{t+1}$ and its derivatives contingent on alternative realizations of Rfree;
    *  Interpreting each of these realizations as a probabilistic possibility
    *  Using these distributions, compute and preserve the resulting $\mathcal{V}_{t+1}(\vec{\aleph})$ and derivatives
1. Use the fact that 

My conclusion is that we should NOT mandate separate solution objects (in the sense of things that are stored sequentially on something like the current `solution[]` list).  




(Below is Matt's email).

On Thu, Jun 24, 2021 at 5:23 PM Matthew White <mnwhite@gmail.com> wrote:
Chris and Seb:

However contentious it got at times, I'm really glad we had today's talk. What Chris proposed / discussed about moving integration / expectation over period t shocks *inside* the "period t solver" is really important and is a major step in moving us toward a more modular structure for HARK that ties its solution and simulation methods together. It's also related to fixing the currently awkward timing of HARK data.

I'm trying to work through something that formalizes the events/frames structure we had previously discussed. Essentially, we want phrase models as a sequence of steps/events/frames, each of which does one *kind* of thing. Types of events would include:

- Attrition and replacement
- A continuously-distributed shock arrives (univariate or multivariate), allowing some new variables to be realized
- A discrete variable is perturbed by a Markov transition (that might be a function of other variables)
- The agent makes a continuous choice over one or more variables, maximizing some function
- The agent makes a discrete choice, maximizing some function

Each event would be self-contained, and would have a representation of both what it means to simulate that event forward and to solve it backward. "One period" of the problem is a list of events that happen in sequence. Here's a conceptual breakdown of what this would look like for ConsIndShock, where I'm defining the "beginning" of period t as the split second before income shocks arrive. I'm using k_t to mean capital holdings, which are the same thing as end-of-period assets for non-newborns.

The big thing is that *every* event has a value function (with associated marginal value functions, or methods for calculating them) over the variables named in "vars in", excluding the ones named in "ignore during soln in". There suffixes _t, _tp1, and _tm1 would also be understood by the code to refer to time indexing.

```
--------------------------------------------------
EVENT 0: 
Death and birthtype: attrition and replacement 
vars in: aNrm_tm1, pLvl_tm1
vars out: kNrm_t, pLvl_temp
ignore during soln in: pLvl_tm1
ignore during soln out: pLvl_temp
params: LivPrb_t
distributions: kNewDstn, pNewDstn
shocks: kNew, pNew
replacement rule: Bernoulli(1.0-LivPrb_t)
terminal value: ConstantFunction(0.0)
transitions old: kNrm_t = aNrm_tm1, pLvl_temp = pLvl_tm1
transitions new: kNrm_t = kNew, pLvl_temp = pNew

--------------------------------------------------

--------------------------------------------------
EVENT 1: Permanent income shock arrives
type: continuous shock realization
vars in: kNrm_t, pLvl_temp
vars out: bNrm_t, pLvl_t
ignore during soln in: pLvl_temp
ignore during soln out: pLvl_t
grids: kXtraGrid_t
shocks: psi
distributions: PermShkDstn_t
params: Rfree_t, PermGroFac_t, CRRA_t
transitions: bNrm_t = Rfree_t * aNrm_tm1 / (psi * PermGroFac_t), pLvl_t = (psi * PermGroFac_t) * pLvl_temp
value: (psi * PermGroFac_t)**(1-CRRA) * V_next(bNrm_t)
marg value: Rfree_t*(psi * PermGroFac_t)**(-CRRA) * V_next.der(bNrm_t)
-----------------------------------------------------

-----------------------------------------------------
EVENT 2: Transitory income shock arrives
type: continuous shock realization  
vars in: bNrm_t
vars out: mNrm_t
grids: bXtraGrid_t
shocks: theta
distributions: TranShkDstn_t
transitions: mNrm_t = bNrm_t + theta

--------------------------------------------------------

-------------------------------------------------------
EVENT 3: Choose consumption
type: continuous choice optimization
vars out: mNrm_t
vars out: aNrm_t
controls: cNrm_t
params: CRRA_t, DiscFac_t, BoroCnstArt_t, aXtraGrid_t
transitions: aNrm_t = mNrm_t - cNrm_t
value: CRRAutility(cNrm_t, gam=CRRA_t) + DiscFac_t * V_next(aNrm_t)
constraints: aNrm_t >= BoroCnstArt_t
solution code: something
--------------------------------------------------------

In this framework, the following value functions would be produced:
V_{t,0}(a_{t-1}) = LivPrb_t * V_{t,1}(k_t = a_{t-1})
V_{t,1}(k_t) = E[ (PermGroFac_t * psi_t)**(1-CRRA_t) * V_{t,2}(b_t = Rfree_t * k_t / (PermGroFac_t * psi_t) ) ]
V_{t,2}(b_t) = E[ V_{t,3}(m_t = b_t + theta_t) ]
V_{t,3}(m_t) = max_{c_t} u(c_t) + DiscFac * V_{t+1,0}(a_t = m_t - c_t)

The traditional Bellman value function is V_{t,3}, defined over the state variable(s) that are relevant when making the control choice at time t. 

The function V_{t,1} is the value of an agent *just before* their income shocks realize; rolling back a split second before that, V_{t,0} is the value of an agent *just before* the mortality shock hits.

It might be that it would be better if what I've called V_{t,0} here should actually be the very *last* thing that happens in a period rather than the first. 

I think that might make starting the simulation easier, and align better with Chris' comments about preferences belonging to "today": how does the individual feel about consumption at t, holding assets at t while surviving, and holding assets at t while dying?
Note that events 0 and 2 don't have a value definition, but 1 and 3 do. This is because the former two events use the obvious default, whereas event 1 has an alternate definition of value due to the renormalization and event 3's value is the maximand of the optimization problem.

     -- mnw

```

Here are a few responses to things in Matt's email:

> "One period" of the problem is a list of events that happen in sequence.
   - By self-contained events, I interpret you as meaning successive entries in the `solution[]` list

Seb and I have had a long exchange of discussions of this.  My view was originally identical to the one you articulte above: We should have a separate self-contained entity that encapsulates separately every single event that happens.

But when we started breaking things down we realized that:

1. It is a burden on the user to turn everything into an event
1. There are way more "events" than are worth separately tracking

So, for example, if we think about the portfolio model, in addition to all the stages you list, there would also be stages for:
1. Calculating the portfolio-weighted rate of return
1. Drawing the risky asset's rate of return shock
1. Computing the expected value functions taking that uncertain return into account
    * Treating the portfolio share choice as a state variable
1. Choosing the optimal portfolio share contingent on those expected values
1. Calculating value functions contingent on the agent having chosen the optimal share

So, this is 8 or 9 solution objects created, which seems excessive and cumbersome.

But the crucial point is that we would ask the user to go to the trouble of creating several solution objects that have two state variables -- portfolio share AND liquid assets -- even though both before and after those steps there is only one state variable, liquid assets.  

I'd argue that it would be a lot easier to handle with the following solution objects:

0. Beginning of period value calculation stage
   * All shocks (including survival, rate of return, income, etc) are viewed as being realized during this stage.
   * Production of the expected value functions is the purpose of the stage
1. Consumption choice stage

Remember that something else on our agenda is to abstract the description of our models into standalone of model-description equations like those Dolo uses.  If we had accomplished that, it's unclear what purpose would be solved by creating a separate object for each individual event.  We'd need an EQUATION or set of equations that corresponds to each event within a period, but not a separate standalone solution object.  

